In [2]:
## mongoDB -> pyspark -> MySQL저장 (수동)

from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
import pyspark.sql.functions as f


# conf = SparkConf()
# conf.set('spark.jars.packages', "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1")
# spark = SparkSession.builder.appName("multi").config(conf=conf).getOrCreate()

spark = SparkSession \
    .builder \
    .appName("multi") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017") \
    .config("spark.mongodb.input.database","datalake") \
    .config("spark.mongodb.input.collection", "i") \
    .config("packages org.mongodb.spark:mongo-spark-connector_2.12:3.0.1").getOrCreate()
sc =spark.sparkContext

# 데이터 스키마
subwaySchema =  StructType([
    StructField("USE_DT", StringType(),True),
    StructField("LINE_NUM", StringType(),True),
    StructField("SUB_STA_NM", StringType(),True),
    StructField("RIDE_PASGR_NUM", IntegerType(),True),
    StructField("ALIGHT_PASGR_NUM", IntegerType(),True),
    StructField("WORK_DT", StringType(),True),
  ])

df = spark.read.schema(subwaySchema).format("com.mongodb.spark.sql.DefaultSource").load() 

In [32]:
dateFormat = "yyyyMMdd"
df.createOrReplaceTempView("df_psg")
df_psg = spark.sql("SELECT * FROM df_psg")
df_psg = df_psg.withColumn("USE_DATE",f.to_date(f.unix_timestamp(df_psg.USE_DT, dateFormat).cast('timestamp')))
df_psg = df_psg.drop('WORK_DT','USE_DT')

df_psg.write \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost/proj") \
    .option("dbtable", "proj.tot_psg") \
    .option("user", "root") \
    .option("password", "1234") \
    .save()


In [31]:
df_psg.select('*').orderBy('USE_DATE',ascending=0).count()

567218

In [ ]:
dateFormat = "yyyyMMdd"
df.createOrReplaceTempView("df2019")
df2019 = spark.sql("SELECT * FROM df2019 WHERE USE_DT LIKE '2019%'")
df2019 = df2019.withColumn("USE_DATE",f.to_date(f.unix_timestamp(df2019.USE_DT, dateFormat).cast('timestamp')))
df2019 = df2019.drop('WORK_DT','USE_DT')

df2019.write \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost/proj") \
    .option("dbtable", "proj.tot2019") \
    .option("user", "root") \
    .option("password", "1234") \
    .save()

df.createOrReplaceTempView("df2020")
df2020 = spark.sql("SELECT * FROM df2020 WHERE USE_DT LIKE '2020%'")
df2020 = df2020.withColumn("USE_DATE",f.to_date(f.unix_timestamp(df2020.USE_DT, dateFormat).cast('timestamp')))
df2020 = df2020.drop('WORK_DT','USE_DT')

df2020.write \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost/proj") \
    .option("dbtable", "proj.tot2020") \
    .option("user", "root") \
    .option("password", "1234") \
    .save()

df.createOrReplaceTempView("df2021")
dateFormat = "yyyyMMdd"
df2021 = spark.sql("SELECT * FROM df2021 WHERE USE_DT LIKE '2021%'")
df2021 = df2021.withColumn("USE_DATE",f.to_date(f.unix_timestamp(df2021.USE_DT, dateFormat).cast('timestamp')))
df2021 = df2021.drop('WORK_DT','USE_DT')


df2021.write \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost/proj") \
    .option("dbtable", "proj.tot2021") \
    .option("user", "root") \
    .option("password", "1234") \
    .save()

In [31]:
# 검색
df.createOrReplaceTempView("df2021")
a = spark.sql("SELECT COUNT(*) FROM df2021 WHERE USE_DT LIKE '20210814'")
a.show()

+--------+
|count(1)|
+--------+
|     601|
+--------+



In [29]:
# pipeline = "{'$match': {'USE_DT': '20210806'}}"
# df = spark.read.format("mongo").option("pipeline", pipeline).load()

# df.show()